In [7]:
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.5 MB/s eta 0:00:00


In [38]:
import math
import numpy as np
import pandas as pd
import Bio
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split
from Bio import Entrez, SeqIO
from Bio.SeqUtils import GC, MeltingTemp
from Bio.SeqUtils import six_frame_translations
from Bio.SeqRecord import SeqRecord
import matplotlib.pyplot as plt


In [16]:
df=pd.read_csv("zoo.csv")

In [30]:
df.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [18]:
df.tail

<bound method NDFrame.tail of     animal_name  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0      aardvark     1         0     0     1         0        0         1   
1      antelope     1         0     0     1         0        0         0   
2          bass     0         0     1     0         0        1         1   
3          bear     1         0     0     1         0        0         1   
4          boar     1         0     0     1         0        0         1   
..          ...   ...       ...   ...   ...       ...      ...       ...   
96      wallaby     1         0     0     1         0        0         0   
97         wasp     1         0     1     0         1        0         0   
98         wolf     1         0     0     1         0        0         1   
99         worm     0         0     1     0         0        0         0   
100        wren     0         1     1     0         1        0         0   

     toothed  backbone  breathes  venomous  fins  legs  t

In [19]:
df.describe()

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
count,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,0.425743,0.198020,0.584158,0.405941,0.237624,0.356436,0.554455,0.603960,0.821782,0.792079,0.079208,0.168317,2.841584,0.742574,0.128713,0.435644,2.831683
std,0.496921,0.400495,0.495325,0.493522,0.427750,0.481335,0.499505,0.491512,0.384605,0.407844,0.271410,0.376013,2.033385,0.439397,0.336552,0.498314,2.102709
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,2.000000
75%,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000,0.000000,1.000000,4.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000,1.000000,1.000000,1.000000,7.000000


In [20]:
df.columns

Index(['animal_name', 'hair', 'feathers', 'eggs', 'milk', 'airborne',
       'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous',
       'fins', 'legs', 'tail', 'domestic', 'catsize', 'class_type'],
      dtype='object')

In [9]:
DNA = Seq("AGTCGCTACGTACAGTACATACTGGTT")
print(DNA + " - Sequence")

AGTCGCTACGTACAGTACATACTGGTT - Sequence


In [10]:
print(DNA.complement() + " - Complement")
print(DNA.reverse_complement() + " - Reverse Complement")

TCAGCGATGCATGTCATGTATGACCAA - Complement
AACCAGTATGTACTGTACGTAGCGACT - Reverse Complement


In [11]:
DNAcount = (DNA.count("A")/len(DNA))
print(DNAcount*100)
print("GC% Percent:\t" + str(GC(DNA)))


25.925925925925924
GC% Percent:	44.44444444444444


/usr/local/lib/python3.10/dist-packages/Bio/SeqUtils/__init__.py:144: BiopythonDeprecationWarning: GC is deprecated; please use gc_fraction instead.
  warnings.warn(


In [13]:
mRNA = DNA.transcribe()
print(mRNA)

AGUCGCUACGUACAGUACAUACUGGUU


In [14]:
protein = mRNA.translate()
print(protein)

SRYVQYILV


In [39]:
def fetch_sequence(accession_number):
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

In [40]:
def sequence_analysis(dna_sequence):
    gc_content = GC(dna_sequence.seq)
    melting_temp = MeltingTemp.Tm_GC(dna_sequence.seq)

    print(f"Sequence ID: {dna_sequence.id}")
    print(f"Sequence length: {len(dna_sequence)}")
    print(f"GC content: {gc_content:.2f}%")
    print(f"Melting temperature: {melting_temp:.2f}°C")

In [41]:
def find_orfs(dna_sequence, min_length=100):
    orfs = []
    for frame, translation in six_frame_translations(dna_sequence.seq):
        for i in range(len(translation)):
            if translation[i:].startswith("M"):
                start = i * 3
                for j in range(i, len(translation)):
                    if translation[j] == "*":
                        end = j * 3 + 3
                        if end - start >= min_length:
                            orfs.append(SeqRecord(dna_sequence.seq[start:end], id=f"ORF_{len(orfs)+1}", description=f"frame_{frame}"))
                        break
    return orfs


In [42]:
def plot_gc_content(dna_sequence):
    window_size = 100
    gc_values = [GC(dna_sequence.seq[i:i + window_size]) for i in range(0, len(dna_sequence), window_size)]
    x_values = [i + window_size / 2 for i in range(0, len(dna_sequence), window_size)]

    plt.plot(x_values, gc_values, label="GC Content")
    plt.xlabel("Position")
    plt.ylabel("GC Content (%)")
    plt.title("GC Content along the DNA Sequence")
    plt.legend()
    plt.show()

In [21]:
x=df.iloc[:,1:17]
y=df.iloc[:,17]

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

#SVM

In [26]:
classifier=svm.SVC()
classifier.fit(x_train,y_train)

SVC()

In [27]:
classifier.predict(x_test[15:25])

array([7, 2, 1, 2, 1, 1, 2, 6, 4, 2])

In [29]:
classifier.score(x_test,y_test)

1.0

In [37]:
classification_rep = classification_report(y_test, y_pred)
print(classification_rep)

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2

    accuracy                           1.00        26
   macro avg       1.00      1.00      1.00        26
weighted avg       1.00      1.00      1.00        26



#Decision Tree Classifier

In [32]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_train, y_train)

DecisionTreeClassifier(random_state=42)

In [34]:
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [35]:
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2

    accuracy                           1.00        26
   macro avg       1.00      1.00      1.00        26
weighted avg       1.00      1.00      1.00        26

